In [356]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
import time

In [357]:
news = pd.read_csv("../data/news.csv")
price = pd.read_csv("../data/stock.csv")

In [359]:
merge = pd.DataFrame(columns=["symbol", "date", "time","content", "preprice", "latprice", "match", "sentiment", "score"])

In [360]:
for title in ["symbol", "date", "time", "content", "match", "sentiment"]:
    merge[title] = news[title]
merge["score"] = news["match"] * news["sentiment"]

In [ ]:

pzmerge = pd.DataFrame()
for idx, new in tqdm(news.iterrows()):
    date = new["date"]
    time = new["time"]
    match =new["match"]
    senti = new["sentiment"]
    cb = date +" "+ time
    usdate = dt.datetime.strptime(cb, '%Y-%m-%d %H:%M:%S')
    usdate -= dt.timedelta(hours=5)
    symbol = new["symbol"].lower()
    symbol = symbol if not symbol == "googl" else "goog"
    dtformat = dt.datetime.strptime(date, '%Y-%m-%d').date()
    while True:
        if price['Date'].eq(date).any():
            break
        else:
            dtformat+=dt.timedelta(days=1)
            date = dt.datetime.strftime(dtformat,'%Y-%m-%d')
    row = price[price["Date"]==date]
    if usdate.hour<16:
        nowprice = price.iloc[row.index][[symbol]]
        preprice = price.iloc[row.index-1][[symbol]]
        nextprice = price.iloc[row.index+1][[symbol]]
    else:
        preprice = price.iloc[row.index][[symbol]]
        nowprice = price.iloc[row.index+1][[symbol]]
        nextprice = price.iloc[row.index+2][[symbol]]
    pz = pd.concat([preprice,nowprice,nextprice]).T
    pz.columns = ["preprice","nowprice","nextprice"]
    pzmerge = pd.concat([pzmerge, pz])


In [334]:
df = news.reset_index(drop=True)
df1 = pzmerge.reset_index(drop=True)
merge = pd.concat([df,df1], axis = 1)
print(merge)
merge.to_csv("../data/merge2.csv")

      symbol        date      time                                  uuid  \
0       META  2022-11-05  20:13:24  d5b8c1b1-a352-488a-99a0-98084f6a425f   
1       META  2022-11-05  18:02:00  186d471c-27a8-4e3d-a905-ab6982b32996   
2       META  2022-11-05  18:00:00  a6e7fdb2-80ff-46ea-bd07-53f5c6416111   
3       META  2022-11-05  16:49:01  5ff55fac-7139-46f1-bc32-30ca00379954   
4       META  2022-11-05  11:58:00  3291b5cf-5c58-4a63-a1c0-db633b9ba218   
...      ...         ...       ...                                   ...   
25273   NFLX  2021-01-22  14:55:08  2e8daa5d-b22e-43de-93b1-03ef99e57857   
25274   NFLX  2021-01-22  14:48:47  df63c55b-0ed3-432c-aa66-69911d9986e7   
25275   NFLX  2021-01-22  14:42:59  a3db3a06-5a21-4dd7-b42b-63fbfbd48a49   
25276   NFLX  2021-01-22  13:55:04  e9447691-d953-4025-8d04-2cdf8aaa0aa7   
25277   NFLX  2021-01-22  13:30:00  68dfe285-a462-4765-97f9-e261b177c24a   

           match  sentiment  \
0      13.746987  -0.458800   
1       9.408848   0.6249